In [3]:
import cv2
import numpy as np

In [2]:
if "google.colab" in str(get_ipython()):
    from google.colab.patches import cv2_imshow

    imshow = cv2_imshow
else:

    def imshow(img):
        cv2.imshow("ImageWindow", img)
        cv2.waitKey()

        cv2.destroyAllWindows()


In [3]:
img1  = cv2.imread('img1.jpg')
img2  = cv2.imread('img2.jpg')
img1 = cv2.resize(img1, None, fx=0.2, fy=0.2)
img2 = cv2.resize(img2, None, fx=0.2, fy=0.2)

In [4]:
img1_gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)

In [83]:
def detect_corners(image):
    image = image.copy()
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    kernel_size = (5, 5)

    blurred_image = cv2.blur(gray, kernel_size)

    sobel_x = cv2.Sobel(blurred_image, cv2.CV_64F, 1, 0, ksize=3)
    sobel_y = cv2.Sobel(blurred_image, cv2.CV_64F, 0, 1, ksize=3)
    magnitude = np.sqrt(sobel_x**2 + sobel_y**2)
    magnitude = np.uint8(magnitude)

    _, thresh = cv2.threshold(magnitude, 30, 255, cv2.THRESH_BINARY)

    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    largest_contour = max(contours, key=cv2.contourArea)
    corner_points = []

    epsilon = 0.02 * cv2.arcLength(largest_contour, True)
    approx = cv2.approxPolyDP(largest_contour, epsilon, True)

    if len(approx) == 4:
        corner_points.extend(approx)

    corner_points = np.array(corner_points)
    return corner_points

    for corner in corner_points:
        x, y = corner.ravel()
        cv2.circle(image, (x, y), 10, 255, -1)

    # cv2.imshow('Corners Detected', image)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
        
    return corner_points

In [43]:
def order_corner_points(corners):
    corners = [(corner[0][0], corner[0][1]) for corner in corners]
    top_r, top_l, bottom_l, bottom_r = corners[0], corners[1], corners[2], corners[3]
    return (top_l, top_r, bottom_r, bottom_l)

In [46]:
def perspective_transform(image, corners):
    ordered_corners = order_corner_points(corners)
    top_l, top_r, bottom_r, bottom_l = ordered_corners

    width_A = np.sqrt(((bottom_r[0] - bottom_l[0]) ** 2) + ((bottom_r[1] - bottom_l[1]) ** 2))
    width_B = np.sqrt(((top_r[0] - top_l[0]) ** 2) + ((top_r[1] - top_l[1]) ** 2))
    width = max(int(width_A), int(width_B))

    height_A = np.sqrt(((top_r[0] - bottom_r[0]) ** 2) + ((top_r[1] - bottom_r[1]) ** 2))
    height_B = np.sqrt(((top_l[0] - bottom_l[0]) ** 2) + ((top_l[1] - bottom_l[1]) ** 2))
    height = max(int(height_A), int(height_B))

    dimensions = np.array([[0, 0], [width - 1, 0], [width - 1, height - 1], 
                    [0, height - 1]], dtype = "float32")

    ordered_corners = np.array(ordered_corners, dtype="float32")

    matrix = cv2.getPerspectiveTransform(ordered_corners, dimensions)

    return cv2.warpPerspective(image, matrix, (width, height))

In [84]:
image = cv2.imread('img4.jpg')

corners = detect_corners(image)
print(len(corners))
cv2.imshow('Original Image', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

transformed_image = perspective_transform(image, corners)
cv2.imshow('Transformed Image', transformed_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

4


In [85]:
video_path = 'vid1.mp4'  # Replace with the path to your video file
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Error: Couldn't open video.")
    exit()

# Get video information
fps = cap.get(cv2.CAP_PROP_FPS)
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Process the video
frame_number = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Process every 5 frames
    if frame_number % 5 == 0:
        # Perform object detection on the current frame
        detected_objects = detect_objects(frame)

        # Draw rectangles and labels on the frame
        for obj in detected_objects:
            x, y, width, height, obj_class = obj
            color = (0, 255, 0)  # Example color, you can customize based on class
            cv2.rectangle(frame, (x, y), (x + width, y + height), color, 2)
            label = f'Class {obj_class}'
            cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        # Display the frame
        cv2.imshow('Object Detection', frame)
        cv2.waitKey(0)  # You can adjust the wait key or use cv2.waitKey(1) for a delay

    frame_number += 1

    if cv2.waitKey(1) & 0xFF == ord('q'):  # Press 'q' to exit
        break

# Release the video capture and close the window
cap.release()
cv2.destroyAllWindows()


NameError: name 'detect_objects' is not defined

## Detecting objects on a photo

In [5]:
def detect_objects(image):

  gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

  sobel_x = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3)
  sobel_y = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3)

  sobel_combined = np.sqrt(sobel_x**2 + sobel_y**2)

  sobel_combined = np.uint8(sobel_combined)

  _, binary_image = cv2.threshold(sobel_combined, np.mean(sobel_combined) * 2, 255, cv2.THRESH_BINARY)
  imshow(binary_image)

  contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

  contour_image = img1.copy()

  valid_contours = [cnt for cnt in contours if cv2.contourArea(cnt) > 1500]

  detected_objects = []

  for contour in valid_contours:
      x, y, w, h = cv2.boundingRect(contour)

      cropped_image = image[y:y + h, x:x + w]
      detected_objects.append(cropped_image)

      imshow( cropped_image)

  return detected_objects

In [6]:
detected_objects = detect_objects(img1)

### Idea is to create templates for matching different objects, so far:
 - Capitol
 - Any non reversed card (template for a card is just a random card)

In [7]:
template1 = cv2.imread('template1.png')
template2 = cv2.imread('template2.png')
templates = [template1, template2]
for template in templates:
  imshow(template)

## Matching templates

Templates are scaled to size of detected object and then each template is matched to each object to return which objects match given template, instead of hard coded value we would like to explore determining objects by most simmilar template given some small threshold is passed to exclude noise

In [8]:
def match_templates(templates, detected_objects):
  matches = []
  for template in templates:
    print(f'Matching template:')
    imshow(template)
    print()
    for i, obj in enumerate(detected_objects):
      template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

      template_resized = cv2.resize(template_gray, (obj.shape[:2][::-1]))
      obj_gray = cv2.cvtColor(obj, cv2.COLOR_BGR2GRAY)
      result = cv2.matchTemplate(obj_gray, template_resized, cv2.TM_CCOEFF_NORMED)
      min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)

      matches.append(max_val)
      threshold = 0.3
      if max_val > threshold:
          # This contour matches template1
          print(f"Match! {i}")
          imshow(obj)
    print('-'*10)

In [9]:
match_templates(templates, detected_objects)

Matching template:

Match! 1
Match! 2
Match! 3
----------
Matching template:

Match! 4
----------


## Problems:
 - there are 6 fractions each having cards in different color
 - there will probably be need to preprocess images to take care of lighting

We thought about 2 ways of dealing with first problem:
1. Based on results below it could be possible to create generic templates(gray) that could match each fraction and if possible exclude art from template so it wouldn't get compared

2. Create 6 sets of templates for each faction. Pretty self explanatory 6 is not a big number this shouldn't increase computational need greatly so maybe this approach would yield better results

In [10]:
detected_objects = detect_objects(img2)

In [11]:
match_templates(templates, detected_objects)

Matching template:

Match! 2
Match! 4
----------
Matching template:

Match! 3


## Conclusions

We believe we could make this method work pretty well on photos, however we are not sure how it generalizes to a video, making computations each frame probbably is quite intense, so we could try detecting objects at start of a video and the classify any new objects that were moved

We would greatly appreciate feedback and any form of guidence to a reasonable approach